- file 1 : 2023-05_260_34P0_in-network-rates.json.gz  (Saved both tables)
- file 2 : 2023-05_266_38B0_in-network-rates_2_of_3.json.gz (saved only Innetwork Table)
- file 3 : 2023-05_266_38F0_in-network-rates_1_of_2.json.gz (saved only Innetwork Table)
- file 4 : 2023-05_290_37B0_in-network-rates.json.gz (saved in both tables)
- file 5 : 2023-05_290_37D0_in-network-rates.json.gz ( saved in both tables)
- file 6 : 2023-05_302_42B0_in-network-rates_1_of_8.json.gz (saved only ProvRef Table)
- file 7 : 2023-05_302_42B0_in-network-rates_2_of_8.json.gz (saved only ProvRef Table)
- file 8 : 2023-05_302_42B0_in-network-rates_3_of_8.json.gz (saved only ProvRef Table)
- file 9 : 2023-05_302_42B0_in-network-rates_4_of_8.json.gz (saved only ProvRef Table)
- file 10 : 2023-05_302_42B0_in-network-rates_5_of_8.json.gz (running)

In [0]:
from pyspark.sql.functions import *
#Get the file name from the driver notebook
# filename = dbutils.notebook.entry_point.getCurrentBindings()
# fname =filename.get('filename')

fname = "dbfs:/mnt/floridablue/may2023/raw/ProviderReferences/2023-05_302_42B0_in-network-rates_5_of_8.json.gz"

In [0]:
# files=['dbfs:/mnt/march2023/ProviderReferences/smallFiles/2023-03_302_42F0_in-network-rates.json.gz']
#Read the file from blob and get the schema
# for file in files:
#try:
strInputFile = fname #'dbfs:/mnt/march2023/ProviderReferences/smallFiles/2023-03_230_30B0_in-network-rates.json.gz'
try:
    dfJSONML = spark.read.option('multiline',True).json(strInputFile)
#         dfJSON.printSchema()
#Get the file name to update into the table
    strfname = strInputFile.split('/')
    inputFileName = strfname[-1] 
    print('The File under process is : ',inputFileName)
except Exception as e:
    print(e)
        #  spark.sql(f'update default.bcbs_florida_blue_may2023 set status="Failed at step 1", process_date = CURRENT_DATE,Remarks="Unable to read the file with multiline option" where fileName like "%{inputFileName}%"')


df_Meta = dfJSONML.select(dfJSONML.reporting_entity_name,dfJSONML.reporting_entity_type,dfJSONML.version, dfJSONML.last_updated_on)
#df_Meta.display()
if 'provider_references' in dfJSONML.columns and 'npi' in dfJSONML.schema.simpleString():
    try:
        df_ProviderRef = dfJSONML.select('provider_references')
        df_ProviderRef = df_ProviderRef.withColumn('provider_references',explode(col('provider_references')))\
                            .withColumn('provider_group_id',col('provider_references.provider_group_id'))\
                            .withColumn('provider_groups',explode(col('provider_references.provider_groups')))\
                            .withColumn('pg_npi',col('provider_groups.npi'))\
                            .withColumn('pg_tin_type',col('provider_groups.tin.type'))\
                            .withColumn('pg_tin_value',col('provider_groups.tin.value'))\
                            .withColumn('sourceFile',lit(input_file_name()))\
                            .withColumn('Created_Date',current_timestamp())\
                            .drop('provider_references','provider_groups')
        print('created Provider ref')
        df_ProviderRef.write.mode('append').saveAsTable('May_tblProviderReference_failed_files_retry')
        print('written table  Provider ref')
        # spark.sql(f'update default.bcbs_florida_blue_may2023 set status="Loaded Privder Ref", process_date = CURRENT_DATE,Remarks="successfully updated provide ref table" where fileName like "%{inputFileName}%"')
    except Exception as e:
        print(e)
            #  spark.sql(f'update default.bcbs_florida_blue_may2023 set status="Failed at step 2", process_date = CURRENT_DATE,Remarks="Unable to load provider References" where fileName like "%{inputFileName}%"')
try:
    dfJSON = spark.read.json(strInputFile)
except Exception as e:
    print(e)
#     spark.sql(f'update default.bcbs_florida_blue_may2023 set status="Failed at step3", process_date = CURRENT_DATE,Remarks="Unable to read the file without multiline option" where fileName like "%{inputFileName}%"')
# #         dfJSON.printSchema()
schema = dfJSON.schema.simpleString()
if 'provider_groups' in dfJSON.columns:
    dfJSON = dfJSON.drop('provider_groups')
if 'provider_references' in dfJSON.columns:
    dfJSON = dfJSON.drop('provider_references')
if 'provider_group_id' in dfJSON.columns:
    dfJSON = dfJSON.drop('provider_group_id')    
if 'bundled_codes' in dfJSON.columns:
    dfJSON = dfJSON.drop('bundled_codes')

if '_corrupt_record' in schema and len(dfJSON.columns)>1:
    print('currupt record found')
    
    dfData = dfJSON.withColumn('exp_NegotiatedRates',explode(col('negotiated_rates')))\
                .drop('negotiated_rates','_corrupt_record')\
                .withColumn('provider_references', explode(col('exp_NegotiatedRates.provider_references')))\
                .withColumn('exp_negotiated_prices',explode(col('exp_NegotiatedRates.negotiated_prices')))\
                    .drop('exp_NegotiatedRates')\
                .withColumn('np_billing_class',col('exp_negotiated_prices.billing_class'))\
                .withColumn('np_expiration_date',col('exp_negotiated_prices.expiration_date'))\
                .withColumn('np_negotiated_rate',col('exp_negotiated_prices.negotiated_rate'))\
                .withColumn('np_negotiated_type',col('exp_negotiated_prices.negotiated_type'))\
                .withColumn('np_service_code',col('exp_negotiated_prices.service_code'))\
                .withColumn('sourceFile',lit(input_file_name()))\
                .withColumn('Created_Date',current_timestamp())\
                .drop('exp_negotiated_prices')

    dfData = dfData.join(broadcast(df_Meta))
    print('joined meta')
    # dfData.printSchema()
        
else:
    print(' no currupt record ')
    dfJSON = dfJSONML
    dfJSON = dfJSON.drop('provider_references')     
    dfData = dfJSON.withColumn('in_network',explode(col('in_network')))\
                .drop('provider_references')\
                .withColumn('billing_code',col('in_network.billing_code'))\
                .withColumn('billing_code_type',col('in_network.billing_code_type'))\
                .withColumn('billing_code_type_version',col('in_network.billing_code_type_version'))\
                .withColumn('name',col('in_network.name'))\
                .withColumn('negotiation_arrangement',col('in_network.negotiation_arrangement'))\
                .withColumn('exp_NegotiatedRates',explode(col('in_network.negotiated_rates')))\
                .drop('negotiated_rates','in_network')\
                .withColumn('provider_references', explode(col('exp_NegotiatedRates.provider_references')))\
                .withColumn('exp_negotiated_prices',explode(col('exp_NegotiatedRates.negotiated_prices')))\
                    .drop('exp_NegotiatedRates')\
                .withColumn('np_billing_class',col('exp_negotiated_prices.billing_class'))\
                .withColumn('np_expiration_date',col('exp_negotiated_prices.expiration_date'))\
                .withColumn('np_negotiated_rate',col('exp_negotiated_prices.negotiated_rate'))\
                .withColumn('np_negotiated_type',col('exp_negotiated_prices.negotiated_type'))\
                .withColumn('np_service_code',col('exp_negotiated_prices.service_code'))\
                .withColumn('sourceFile',lit(input_file_name()))\
                .withColumn('Created_Date',current_timestamp())\
                .drop('exp_negotiated_prices')
    # dfData.printSchema()
dfData = dfData.withColumn('np_service_code',explode(col('np_service_code')))
# #Saving the innetwork details into table
dfData = dfData.where(dfData.np_service_code.isin('-1','02','11','20','21','22','23','31','32','34','51','61'))
dfData= dfData.withColumn('np_negotiated_rate',dfData.np_negotiated_rate.cast('double'))
try:
    dfData.write.mode('append').option('mergeSchema',True).saveAsTable('may_tblProviderReferenceInNetwork_Mano')
    print('written to  Provider references innetwork')
#     spark.sql(f'update default.bcbs_florida_blue_may2023 set status="Loaded both Provider references and innetwork", process_date = CURRENT_DATE,Remarks="Both atables loaded" where fileName like "%{inputFileName}%"')
except Exception as e:
    print(e)
#     spark.sql(f'update default.bcbs_florida_blue_may2023 set status="Failed at step 4", process_date = CURRENT_DATE,Remarks="Unable to load into provider references innetwork" where fileName like "%{inputFileName}%"')

The File under process is :  2023-05_302_42B0_in-network-rates_5_of_8.json.gz
created Provider ref
written table  Provider ref


In [0]:
dfJSONML.printSchema()

root
 |-- in_network: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- billing_code: string (nullable = true)
 |    |    |-- billing_code_type: string (nullable = true)
 |    |    |-- billing_code_type_version: string (nullable = true)
 |    |    |-- bundled_codes: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- billing_code: string (nullable = true)
 |    |    |    |    |-- billing_code_type: string (nullable = true)
 |    |    |    |    |-- billing_code_type_version: string (nullable = true)
 |    |    |    |    |-- description: string (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- negotiated_rates: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- negotiated_prices: array (nullable = true)
 |    |    |    |    |    |-- element: struct (containsNull

In [0]:
# %sql
# select DISTINCT (sourceFile) AS sourceFile from tblProviderReferenceInNetworkmay -- where sourcefile in ('dbfs:/mnt/march2023/ProviderReferences/smallFiles/2023-03_302_42F0_in-network-rates.json.gz')

In [0]:
# dbutils.notebook.exit("File Execution is successfully completed")